In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False
import warnings
warnings.filterwarnings(action='ignore') 

In [2]:
backtest_all=pd.read_csv("./데이터/일일주식데이터.csv",encoding='cp949',index_col=0)
data = pd.read_csv("./데이터/5. label 지정 완료.csv", encoding='cp949')
feature = pd.read_csv("./데이터/8-2. Std feature.csv")
data

,회사명,거래소코드,회계년도,상장일,상장폐지일,소속코드,산업명,매출액증가율,유형자산증가율,재고자산증가율,...,당좌자산회전률,부가가치,총자본투자효율,설비투자효율,기계투자효율,부가가치율,노동소득분배율,자본분배율,이윤분배율,label
0,(주)CMG제약,58820,2011-12-01,2001-08-31,NaN,5,의료용 물질 및 의약품 제조업,56.40,-5.90,-1.43,...,1.55,3472.17,12.72,35.86,128.51,19.56,89.82,10.18,-43.44,0.0
1,(주)CMG제약,58820,2012-12-01,2001-08-31,NaN,5,의료용 물질 및 의약품 제조업,-8.43,-4.12,7.73,...,0.57,269.31,0.47,2.89,10.67,1.66,1749.87,-1649.87,-2473.07,0.0
2,(주)CMG제약,58820,2013-12-01,2001-08-31,NaN,5,의료용 물질 및 의약품 제조업,25.51,16.09,56.29,...,0.52,6994.90,12.51,77.91,289.40,34.29,71.28,28.72,4.23,0.0
3,(주)CMG제약,58820,2014-12-01,2001-08-31,NaN,5,의료용 물질 및 의약품 제조업,11.52,74.30,6.37,...,0.80,7061.34,12.16,37.80,132.06,31.04,79.30,20.70,2.24,0.0
4,(주)CMG제약,58820,2015-12-01,2001-08-31,NaN,5,의료용 물질 및 의약품 제조업,18.85,-5.37,11.04,...,1.22,4315.37,7.63,24.21,90.92,15.96,144.10,-44.10,-84.58,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9273,흥구석유(주),24060,2016-12-01,1994-12-07,NaN,5,도매 및 상품 중개업,-7.26,-5.61,52.46,...,2.20,13728.30,10.92,102.82,3801.25,6.86,44.46,55.54,43.99,0.0
9274,흥구석유(주),24060,2017-12-01,1994-12-07,NaN,5,도매 및 상품 중개업,8.12,-0.56,9.41,...,1.78,14659.03,11.36,118.51,5704.66,9.22,43.35,56.65,43.02,0.0
9275,흥구석유(주),24060,2018-12-01,1994-12-07,NaN,5,도매 및 상품 중개업,6.87,-0.36,-46.09,...,1.76,14287.12,10.80,118.81,4046.12,8.82,46.23,53.77,40.99,0.0
9276,흥구석유(주),24060,2019-12-01,1994-12-07,NaN,5,도매 및 상품 중개업,-9.35,-7.00,96.40,...,2.18,11012.82,8.64,92.04,2603.45,5.61,62.14,37.86,27.35,0.0


In [3]:
# 거래소 코드 6개로 맞춰주기
for i in data.index:
    if len(str(data.loc[i,"거래소코드"])) == 3:
        data.loc[i,"거래소코드"]='000'+str(data.loc[i,"거래소코드"])
    elif len(str(data.loc[i,"거래소코드"])) == 4:
        data.loc[i,"거래소코드"]='00'+str(data.loc[i,"거래소코드"])
    elif len(str(data.loc[i,"거래소코드"])) == 5:
        data.loc[i,"거래소코드"]='0'+str(data.loc[i,"거래소코드"])
    elif len(str(data.loc[i,"거래소코드"])) == 2:
        data.loc[i,"거래소코드"]='0000'+str(data.loc[i,"거래소코드"])

# 기업의 규모를 고려 년초기준 시가총액 기준 20%이하는 제거

In [4]:
from pykrx import stock
from pykrx import bond

In [5]:
fea = feature["ela_feature"].tolist()
fea 

['자기자본증가율',
 '총자본증가율',
 '매출액순이익률',
 '자기자본순이익률',
 '총자본사업이익률',
 '총자본순이익률',
 '순운전자본비율',
 '자기자본구성비율',
 '자기자본배율',
 '매출채권회전기간',
 '순운전자본회전률',
 '부가가치율']

In [6]:
data.columns

Index(['회사명', '거래소코드', '회계년도', '상장일', '상장폐지일', '소속코드', '산업명', '매출액증가율',
       '유형자산증가율', '재고자산증가율', '비유동자산증가율', '유동자산증가율', '자기자본증가율', '총자본증가율',
       '순이익증가율', '매출액총이익률', '매출액순이익률', '자기자본순이익률', '총자본사업이익률', '총자본순이익률',
       '1주당매출액', '1주당순이익', '유동비율', '당좌비율', '현금비율', '순운전자본비율', '부채비율',
       '이자보상배율이자비용', '비유동비율', '유동자산구성비율', '자기자본구성비율', '자기자본배율', '총자본회전률',
       '자기자본회전률', '비유동자산회전률', '재고자산회전률', '매출채권회전률', '매입채무회전률', '매입채무회전기간',
       '재고자산회전기간', '매출채권회전기간', '1회전기간', '운전자본회전률', '순운전자본회전률', '유형자산회전율',
       '유동자산회전률', '당좌자산회전률', '부가가치', '총자본투자효율', '설비투자효율', '기계투자효율', '부가가치율',
       '노동소득분배율', '자본분배율', '이윤분배율', 'label'],
      dtype='object')

In [7]:
data = data[['회사명', '거래소코드', '회계년도', '상장일', '상장폐지일', '소속코드', '산업명','자기자본증가율',
 '총자본증가율', '매출액순이익률', '자기자본순이익률', '총자본사업이익률', '총자본순이익률', '순운전자본비율', '자기자본구성비율',
 '자기자본배율', '매출채권회전기간', '순운전자본회전률', '부가가치율','label']]

In [8]:
data["상장일"]=pd.to_datetime(data['상장일'])
data["회계년도"] = pd.to_datetime(data["회계년도"])
data["투자년도"] = data["회계년도"].dt.year
data["투자년도"]

0       2011
1       2012
2       2013
3       2014
4       2015
        ... 
9273    2016
9274    2017
9275    2018
9276    2019
9277    2020
Name: 투자년도, Length: 9278, dtype: int64

In [9]:
data

,회사명,거래소코드,회계년도,상장일,상장폐지일,소속코드,산업명,자기자본증가율,총자본증가율,매출액순이익률,...,총자본사업이익률,총자본순이익률,순운전자본비율,자기자본구성비율,자기자본배율,매출채권회전기간,순운전자본회전률,부가가치율,label,투자년도
0,(주)CMG제약,058820,2011-12-01,2001-08-31,NaN,5,의료용 물질 및 의약품 제조업,-2.02,1.98,-8.50,...,2.52,-5.58,28.23,66.05,82.55,148.80,0.86,19.56,0.0,2011
1,(주)CMG제약,058820,2012-12-01,2001-08-31,NaN,5,의료용 물질 및 의약품 제조업,75.17,109.15,-40.97,...,-9.65,-15.78,70.60,55.32,95.69,197.16,0.42,1.66,0.0,2012
2,(주)CMG제약,058820,2013-12-01,2001-08-31,NaN,5,의료용 물질 및 의약품 제조업,55.10,-2.09,1.45,...,2.81,0.52,58.50,87.63,113.12,176.20,0.42,34.29,0.0,2013
3,(주)CMG제약,058820,2014-12-01,2001-08-31,NaN,5,의료용 물질 및 의약품 제조업,0.01,3.86,0.69,...,1.25,0.28,33.72,84.39,113.13,196.72,0.56,31.04,0.0,2014
4,(주)CMG제약,058820,2015-12-01,2001-08-31,NaN,5,의료용 물질 및 의약품 제조업,-4.48,-2.64,-13.50,...,-4.30,-6.37,30.82,82.80,106.02,174.14,0.75,15.96,0.0,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9273,흥구석유(주),024060,2016-12-01,1994-12-07,NaN,5,도매 및 상품 중개업,1.39,1.39,3.02,...,4.67,4.75,63.26,79.10,1008.27,66.81,1.53,6.86,0.0,2016
9274,흥구석유(주),024060,2017-12-01,1994-12-07,NaN,5,도매 및 상품 중개업,0.83,0.98,3.97,...,4.92,4.95,58.09,80.35,1052.03,64.90,1.28,9.22,0.0,2017
9275,흥구석유(주),024060,2018-12-01,1994-12-07,NaN,5,도매 및 상품 중개업,0.99,-5.89,3.61,...,4.27,4.48,60.65,81.76,1096.89,65.07,1.32,8.82,0.0,2018
9276,흥구석유(주),024060,2019-12-01,1994-12-07,NaN,5,도매 및 상품 중개업,9.31,11.24,1.53,...,4.02,2.32,59.10,83.71,1082.55,55.54,1.65,5.61,0.0,2019


In [10]:
def mod(backtest_all,data, y):
    backtest_all["Date"]=pd.to_datetime(backtest_all["Date"])
    backtest_y = backtest_all[backtest_all["Date"].dt.year ==y]
    backtest_y.reset_index(drop='index',inplace=True)
    backtest_y["날짜"]=backtest_y["Date"].astype('str').str.replace('-','')
    
    df_y = stock.get_market_cap(backtest_y["날짜"][0],"KOSDAQ")
    a_y  = np.percentile(df_y["시가총액"],q=20)
    total_y = df_y[df_y["시가총액"]>= a_y]
    total_y.reset_index(inplace=True)
    total_y["거래소코드"] = total_y["티커"]

    data_y= data[data["투자년도"]==y]
    data_y = pd.merge(data_y, total_y[["거래소코드","티커"]], how='left', on ='거래소코드')
    data_y = data_y.dropna(subset=['티커']).reset_index(drop=True)
    return backtest_y, data_y

In [11]:
data_2011 = mod(backtest_all, data, 2011)[1]
data_2012 = mod(backtest_all, data, 2012)[1]
data_2013 = mod(backtest_all, data, 2013)[1]
data_2014 = mod(backtest_all, data, 2014)[1]
data_2015 = mod(backtest_all, data, 2015)[1]
data_2016 = mod(backtest_all, data, 2016)[1]
data_2017 = mod(backtest_all, data, 2017)[1]
data_2018 = mod(backtest_all, data, 2018)[1]
data_2019 = mod(backtest_all, data, 2019)[1]
data_2020 = mod(backtest_all, data, 2020)[1]

In [12]:
backtest_2011 = mod(backtest_all, data, 2011)[0]
backtest_2012 = mod(backtest_all, data, 2012)[0]
backtest_2013 = mod(backtest_all, data, 2013)[0]
backtest_2014 = mod(backtest_all, data, 2014)[0]
backtest_2015 = mod(backtest_all, data, 2015)[0]
backtest_2016 = mod(backtest_all, data, 2016)[0]
backtest_2017 = mod(backtest_all, data, 2017)[0]
backtest_2018 = mod(backtest_all, data, 2018)[0]
backtest_2019 = mod(backtest_all, data, 2019)[0]
backtest_2020 = mod(backtest_all, data, 2020)[0]

In [13]:
data_list = [data_2011,data_2012,data_2013,data_2014,data_2015,data_2016,data_2017,data_2018,data_2019,data_2020]

In [14]:
backtest_list = [backtest_2011,backtest_2012,backtest_2013,backtest_2014,backtest_2015,backtest_2016,backtest_2017,backtest_2018,backtest_2019,backtest_2020]

In [15]:
for d in data_list:
    d["총점수"] = 0
    for f in fea:
        if f != '매출채권회전기간':
            per = np.percentile(d[f],q=[10,20,30,40,50,60,70,80,90,100])
            k = 1
            for j in range(len(per)):
                if j == 0:
                    index = d[per[j]>=d[f]].index
                    for m in index:
                        d.loc[m,f+str('_점수')] = k
                       
                elif j != 0:
                    index = d[(per[j-1] < d[f]) & (d[f] <= per[j])].index
                    for m in index:
                        
                        d.loc[m,f+str('_점수')] = k
                       
                k +=1          
            
        else:
            per = np.percentile(d[f],q=[10,20,30,40,50,60,70,80,90,100])
            k = 10
            for j in range(len(per)):
                if j == 0:
                    index = d[per[j]>=d[f]].index
                    for m in index:
                        d.loc[m,f+str('_점수')] = k
                        
                elif j != 0:
                    index = d[(per[j-1] < d[f]) & (d[f] <= per[j])].index
                    for m in index:
                        
                        d.loc[m,f+str('_점수')] = k
                
                k -=1          
    d["총점수"] = d[['자기자본증가율_점수', '총자본증가율_점수', '매출액순이익률_점수', '자기자본순이익률_점수',
       '총자본사업이익률_점수', '총자본순이익률_점수', '순운전자본비율_점수', '자기자본구성비율_점수', '자기자본배율_점수',
       '매출채권회전기간_점수', '순운전자본회전률_점수', '부가가치율_점수']].sum(axis='columns')       

In [16]:
data_all = pd.concat(data_list)
data_all.columns

Index(['회사명', '거래소코드', '회계년도', '상장일', '상장폐지일', '소속코드', '산업명', '자기자본증가율',
       '총자본증가율', '매출액순이익률', '자기자본순이익률', '총자본사업이익률', '총자본순이익률', '순운전자본비율',
       '자기자본구성비율', '자기자본배율', '매출채권회전기간', '순운전자본회전률', '부가가치율', 'label', '투자년도',
       '티커', '총점수', '자기자본증가율_점수', '총자본증가율_점수', '매출액순이익률_점수', '자기자본순이익률_점수',
       '총자본사업이익률_점수', '총자본순이익률_점수', '순운전자본비율_점수', '자기자본구성비율_점수', '자기자본배율_점수',
       '매출채권회전기간_점수', '순운전자본회전률_점수', '부가가치율_점수'],
      dtype='object')

In [17]:
data_2011.sort_values("총점수",ascending=False)[:10]

,회사명,거래소코드,회계년도,상장일,상장폐지일,소속코드,산업명,자기자본증가율,총자본증가율,매출액순이익률,...,매출액순이익률_점수,자기자본순이익률_점수,총자본사업이익률_점수,총자본순이익률_점수,순운전자본비율_점수,자기자본구성비율_점수,자기자본배율_점수,매출채권회전기간_점수,순운전자본회전률_점수,부가가치율_점수
422,메가스터디(주),072870,2011-12-01,2004-12-21,NaN,5,출판업,19.20,16.46,20.72,...,10.0,10.0,10.0,10.0,8.0,9.0,10.0,10.0,6.0,9.0
154,(주)아즈텍더블유비이,032080,2011-12-01,2000-10-26,NaN,5,섬유제품 제조업; 의복제외,16.68,33.66,11.70,...,9.0,9.0,10.0,10.0,10.0,10.0,9.0,6.0,9.0,9.0
89,(주)빅솔론,093190,2011-12-01,2007-08-20,NaN,5,"전자부품, 컴퓨터, 영상, 음향 및 통신장비 제조업",20.79,16.71,15.14,...,10.0,10.0,10.0,10.0,10.0,9.0,7.0,10.0,7.0,9.0
180,(주)에이스침대,003800,2011-12-01,1996-05-03,NaN,6,가구 제조업,15.54,15.72,20.13,...,10.0,9.0,10.0,10.0,6.0,10.0,10.0,10.0,9.0,9.0
273,(주)제주반도체,080220,2011-12-01,2005-02-01,NaN,5,"전자부품, 컴퓨터, 영상, 음향 및 통신장비 제조업",39.07,24.58,15.90,...,10.0,10.0,10.0,10.0,10.0,10.0,4.0,8.0,9.0,7.0
291,(주)컴투스홀딩스,063080,2011-12-01,2009-07-30,NaN,5,출판업,27.86,31.83,37.66,...,10.0,10.0,10.0,10.0,10.0,10.0,10.0,6.0,2.0,10.0
405,대한광통신(주),010170,2011-12-01,1994-11-07,NaN,5,전기장비 제조업,17.50,35.86,39.27,...,10.0,10.0,10.0,10.0,7.0,10.0,10.0,6.0,6.0,10.0
341,(주)티브로드한빛방송,043890,2011-12-01,2000-08-10,2012-11-01,4,방송업,27.16,23.80,19.58,...,10.0,10.0,10.0,10.0,7.0,9.0,3.0,10.0,10.0,9.0
288,(주)카카오,035720,2011-12-01,1999-11-11,NaN,5,정보서비스업,28.06,21.26,25.63,...,10.0,10.0,10.0,10.0,9.0,9.0,10.0,6.0,5.0,8.0
238,(주)유진테크,084370,2011-12-01,2006-01-17,NaN,5,기타 기계 및 장비 제조업,45.79,78.23,19.66,...,10.0,10.0,10.0,10.0,9.0,6.0,7.0,8.0,6.0,9.0


In [18]:
backtest_all = pd.concat(backtest_list)
backtest_all.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Change', '거래소코드',
       '회사명', '날짜'],
      dtype='object')

In [19]:
# data_all.to_csv("./데이터/2. 선정기업자료.csv",encoding='cp949')
# backtest_all.to_csv("./데이터/2. 주식데이터.csv",encoding='cp949')

In [20]:
a = data_2011.sort_values("총점수",ascending=False)[:10]

In [21]:
backtest_2011

,Date,Open,High,Low,Close,Volume,Change,거래소코드,회사명,날짜
0,2011-01-03,1426,1440,1392,1394,439492,-0.023125,58820,(주)CMG제약,20110103
1,2011-01-04,1417,1417,1325,1329,671688,-0.046628,58820,(주)CMG제약,20110104
2,2011-01-05,1333,1351,1260,1321,470690,-0.006020,58820,(주)CMG제약,20110105
3,2011-01-06,1287,1333,1276,1333,422197,0.009084,58820,(주)CMG제약,20110106
4,2011-01-07,1300,1363,1287,1319,167534,-0.010503,58820,(주)CMG제약,20110107
...,...,...,...,...,...,...,...,...,...,...
226704,2011-12-23,1650,1650,1620,1620,18814,-0.009174,24060,흥구석유(주),20111223
226705,2011-12-26,1600,1650,1600,1640,12016,0.012346,24060,흥구석유(주),20111226
226706,2011-12-27,1650,1700,1560,1615,72630,-0.015244,24060,흥구석유(주),20111227
226707,2011-12-28,1590,1680,1520,1530,21185,-0.052632,24060,흥구석유(주),20111228


In [22]:
c = pd.merge(backtest_2011, a, how='right',on ='회사명')
c

,Date,Open,High,Low,Close,Volume,Change,거래소코드_x,회사명,날짜,...,매출액순이익률_점수,자기자본순이익률_점수,총자본사업이익률_점수,총자본순이익률_점수,순운전자본비율_점수,자기자본구성비율_점수,자기자본배율_점수,매출채권회전기간_점수,순운전자본회전률_점수,부가가치율_점수
0,2011-01-03,30001,31030,30001,30744,16266,0.034072,72870,메가스터디(주),20110103,...,10.0,10.0,10.0,10.0,8.0,9.0,10.0,10.0,6.0,9.0
1,2011-01-04,30743,31874,30726,31739,24879,0.032364,72870,메가스터디(주),20110104,...,10.0,10.0,10.0,10.0,8.0,9.0,10.0,10.0,6.0,9.0
2,2011-01-05,31739,32397,31705,32161,26124,0.013296,72870,메가스터디(주),20110105,...,10.0,10.0,10.0,10.0,8.0,9.0,10.0,10.0,6.0,9.0
3,2011-01-06,32211,32346,31536,32330,26506,0.005255,72870,메가스터디(주),20110106,...,10.0,10.0,10.0,10.0,8.0,9.0,10.0,10.0,6.0,9.0
4,2011-01-07,32329,32363,32076,32364,35902,0.001052,72870,메가스터디(주),20110107,...,10.0,10.0,10.0,10.0,8.0,9.0,10.0,10.0,6.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2475,2011-12-23,13341,13341,12756,12845,45900,-0.020214,84370,(주)유진테크,20111223,...,10.0,10.0,10.0,10.0,9.0,6.0,7.0,8.0,6.0,9.0
2476,2011-12-26,12669,13106,12669,12875,37674,0.002336,84370,(주)유진테크,20111226,...,10.0,10.0,10.0,10.0,9.0,6.0,7.0,8.0,6.0,9.0
2477,2011-12-27,12785,13195,12785,13139,71766,0.020505,84370,(주)유진테크,20111227,...,10.0,10.0,10.0,10.0,9.0,6.0,7.0,8.0,6.0,9.0
2478,2011-12-28,13526,14790,13348,13951,235042,0.061801,84370,(주)유진테크,20111228,...,10.0,10.0,10.0,10.0,9.0,6.0,7.0,8.0,6.0,9.0


In [25]:
c.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Change', '거래소코드_x',
       '회사명', '날짜', '거래소코드_y', '회계년도', '상장일', '상장폐지일', '소속코드', '산업명',
       '자기자본증가율', '총자본증가율', '매출액순이익률', '자기자본순이익률', '총자본사업이익률', '총자본순이익률',
       '순운전자본비율', '자기자본구성비율', '자기자본배율', '매출채권회전기간', '순운전자본회전률', '부가가치율',
       'label', '투자년도', '티커', '총점수', '자기자본증가율_점수', '총자본증가율_점수', '매출액순이익률_점수',
       '자기자본순이익률_점수', '총자본사업이익률_점수', '총자본순이익률_점수', '순운전자본비율_점수', '자기자본구성비율_점수',
       '자기자본배율_점수', '매출채권회전기간_점수', '순운전자본회전률_점수', '부가가치율_점수'],
      dtype='object')

In [24]:
c.iloc[:,4:]

,Close,Volume,Change,거래소코드_x,회사명,날짜,거래소코드_y,회계년도,상장일,상장폐지일,...,매출액순이익률_점수,자기자본순이익률_점수,총자본사업이익률_점수,총자본순이익률_점수,순운전자본비율_점수,자기자본구성비율_점수,자기자본배율_점수,매출채권회전기간_점수,순운전자본회전률_점수,부가가치율_점수
0,30744,16266,0.034072,72870,메가스터디(주),20110103,072870,2011-12-01,2004-12-21,NaN,...,10.0,10.0,10.0,10.0,8.0,9.0,10.0,10.0,6.0,9.0
1,31739,24879,0.032364,72870,메가스터디(주),20110104,072870,2011-12-01,2004-12-21,NaN,...,10.0,10.0,10.0,10.0,8.0,9.0,10.0,10.0,6.0,9.0
2,32161,26124,0.013296,72870,메가스터디(주),20110105,072870,2011-12-01,2004-12-21,NaN,...,10.0,10.0,10.0,10.0,8.0,9.0,10.0,10.0,6.0,9.0
3,32330,26506,0.005255,72870,메가스터디(주),20110106,072870,2011-12-01,2004-12-21,NaN,...,10.0,10.0,10.0,10.0,8.0,9.0,10.0,10.0,6.0,9.0
4,32364,35902,0.001052,72870,메가스터디(주),20110107,072870,2011-12-01,2004-12-21,NaN,...,10.0,10.0,10.0,10.0,8.0,9.0,10.0,10.0,6.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2475,12845,45900,-0.020214,84370,(주)유진테크,20111223,084370,2011-12-01,2006-01-17,NaN,...,10.0,10.0,10.0,10.0,9.0,6.0,7.0,8.0,6.0,9.0
2476,12875,37674,0.002336,84370,(주)유진테크,20111226,084370,2011-12-01,2006-01-17,NaN,...,10.0,10.0,10.0,10.0,9.0,6.0,7.0,8.0,6.0,9.0
2477,13139,71766,0.020505,84370,(주)유진테크,20111227,084370,2011-12-01,2006-01-17,NaN,...,10.0,10.0,10.0,10.0,9.0,6.0,7.0,8.0,6.0,9.0
2478,13951,235042,0.061801,84370,(주)유진테크,20111228,084370,2011-12-01,2006-01-17,NaN,...,10.0,10.0,10.0,10.0,9.0,6.0,7.0,8.0,6.0,9.0


In [23]:
c["회사명"].unique()

array(['메가스터디(주)', '(주)아즈텍더블유비이', '(주)빅솔론', '(주)에이스침대', '(주)제주반도체',
       '(주)컴투스홀딩스', '대한광통신(주)', '(주)티브로드한빛방송', '(주)카카오', '(주)유진테크'],
      dtype=object)